In [33]:
import requests
import pandas as pd
import json

def search_recording(artist_name, release_title):
    base_url = "https://musicbrainz.org/ws/2/release/"
    params = {
        "query": f"artist:\"{artist_name}\" AND release:\"{release_title}\"",
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"} # Replace with your app info

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status() # Raise an exception for bad status codes
    data = response.json()
    #print(data)
    
    if data and data.get('releases'):
        # Return the MBID of the first matching recording
        return data['releases'][1]['id']
    return None

# Example usage:
artist = "Steely Dan"
album = "Aja"
release_mbid = search_recording(artist, album)
print(f"Recording MBID for '{album}' by '{artist}': {release_mbid}")

Recording MBID for 'Aja' by 'Steely Dan': cb3d5f81-a4cf-4789-b3bf-724fe75ef1c8


In [34]:
def get_credits_for_recording(release_mbid):
    base_url = f"https://musicbrainz.org/ws/2/release/{release_mbid}"
    params = {
        "inc": "artist-credits+labels+recordings+recording-level-rels+work-rels+work-level-rels+artist-rels + release-groups",  # Include artist relationships
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"}

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()
    #print(data)

    credits = []
    
    if data and data.get('artist-credit'):
       for track in data['media'][0]['tracks']:
            for credit in track['recording']['relations']:
                if credit['attributes'] == []:
                    attributes = 'Not Found'
                else:
                    attributes = credit['attributes']
                credits.append((track['number'], track['title'], credit.get('artist', {}).get('name', 'Not Found'), attributes))
            for credit in track['artist-credit']:
                credits.append((track['number'], track['title'], credit.get('name', 'Not Found'), credit.get('attributes', 'Not Found')))
    if data['relations']:
       for credit in data['relations']:
           trackNum = 'Not Found'
           trackTitle = 'Not Found'
           if credit['attributes'] == [] or not credit['attributes']:
               attributes = 'Not Found'
           else:
               attributes = credit['attributes']
           credits.append((trackNum, trackTitle, credit.get('artist', {}).get('name', 'Not Found'), attributes))
    #print(credits)

    #dataTest = data['media'][0]['tracks'][0]['recording']['relations'][2]['artist']['name']
    #print(dataTest)

    #print("\n#################################################################################################################################\n")

    #credits = [{"Artist" : credit.get('artist', {}).get('name', 'Not Found'), "Role" : credit['attributes'], "Track Number" : track['number'], "Track Title" : track['title']} for track in data['media'][0]['tracks'] for credit in track['recording']['relations']]
    
    return credits

In [35]:
#Example usage:
if release_mbid:
    albumCredits = get_credits_for_recording(release_mbid)
    #print(albumCredits)
else:
    print(f"Recording for '{album}' by '{artist}' not found.")

# Create the DataFrame
df = pd.DataFrame(albumCredits, columns = ['Track Number', 'Track Title', 'Artist', 'Role'])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


# Display the DataFrame
df

,Track Number,Track Title,Artist,Role
0,1,Black Cow,Tom Scott,[horn]
1,1,Black Cow,Roger Nichols,[executive]
2,1,Black Cow,Larry Carlton,[guitar]
3,1,Black Cow,Donald Fagen,[synthesizer]
4,1,Black Cow,Victor Feldman,[Rhodes piano]
5,1,Black Cow,Paul Humphrey,[drums (drum set)]
6,1,Black Cow,Chuck Rainey,[bass guitar]
7,1,Black Cow,Joe Sample,[clavinet]
8,1,Black Cow,Tom Scott,[tenor saxophone]
9,1,Black Cow,Tom Scott,[horn]
